<a href="https://colab.research.google.com/github/bartek717/projectx/blob/main/aeslc_kcentergreedy_FLAN_T5_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, BertTokenizer, BertModel
from datasets import load_dataset, Dataset
import torch
import pandas as pd
from google.colab import files
import time
import math
import os
# here you can change the model if you want. Looks like the available options are (flan-t5-) small, base, large, xl, xxl. Probably easiest to use one of the smaller models for our use case.
!pip install tensorflow-datasets
import tensorflow_datasets as tfds


from google.colab import drive
drive.mount('/content/drive')


  Using cached datasets-2.16.1-py3-none-any.whl (507 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached multiprocess-0.70.15-py310-none-any.whl (134 kB)
Mounted at /content/drive


In [ ]:
import tensorflow_datasets as tfds

# Optionally force redownload of the dataset
dataset = load_dataset("aeslc")

# Access different splits of the dataset
train_dataset = dataset['train']
test_dataset = dataset['test']
validation_dataset = dataset['validation']

# Display some sample data
print(train_dataset[0])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14436 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1960 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1906 [00:00<?, ? examples/s]

{'email_body': 'Greg/Phillip,  Attached is the Grande Communications Service Agreement.\nThe business points can be found in Exhibit C.  I Can get the Non-Disturbance agreement after it has been executed by you and Grande.\nI will fill in the Legal description of the property one I have received it.\nPlease execute and send to:  Grande Communications, 401 Carlson Circle, San Marcos Texas, 78666 Attention Hunter Williams.\n<<Bishopscontract.doc>>\n', 'subject_line': 'Service Agreement'}


In [ ]:
email_bodies = [item['email_body'] for item in train_dataset]
subject_lines = [item['subject_line'] for item in train_dataset]

texts = email_bodies
summaries = subject_lines

import re

disclaimer_pattern = r"This message is for the designated recipient only.*?If you have received it in error.*?Any other use of the email by you is prohibited\."
cleaned_texts = [re.sub(disclaimer_pattern, '', email, flags=re.DOTALL) for email in texts]

texts=cleaned_texts


In [ ]:
# BEGINNING OF CORESET SELECTION

In [ ]:
# IDDS AL paper also uses BERT for embedding/encoding
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_bert = model_bert.to(device)

def encode_texts(texts, batch_size=256):
    encoded_texts = []
    model_bert.eval()  # Set the model to evaluation mode
    for i in range(0, len(texts), batch_size):
        print(f"Processing batch starting at index {i}")
        batch_texts = texts[i:i + batch_size]
        if not batch_texts:
            continue  # Skip empty batches

        inputs = tokenizer_bert(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512, return_token_type_ids=False)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model_bert(**inputs)
        pooled_output = outputs.pooler_output
        encoded_texts.extend(pooled_output.cpu().numpy())

    return encoded_texts

encoded_texts = encode_texts(texts)
encoded_summaries = encode_texts(summaries)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Processing batch starting at index 0
Processing batch starting at index 256
Processing batch starting at index 512
Processing batch starting at index 768
Processing batch starting at index 1024
Processing batch starting at index 1280
Processing batch starting at index 1536
Processing batch starting at index 1792
Processing batch starting at index 2048
Processing batch starting at index 2304
Processing batch starting at index 2560
Processing batch starting at index 2816
Processing batch starting at index 3072
Processing batch starting at index 3328
Processing batch starting at index 3584
Processing batch starting at index 3840
Processing batch starting at index 4096
Processing batch starting at index 4352
Processing batch starting at index 4608
Processing batch starting at index 4864
Processing batch starting at index 5120
Processing batch starting at index 5376
Processing batch starting at index 5632
Processing batch starting at index 5888
Processing batch starting at index 6144
Proces

In [ ]:
import pickle

# Saving encoded_texts
with open('/content/drive/My Drive/encoded_texts.pkl', 'wb') as f:
    pickle.dump(encoded_texts, f)

# Saving encoded_summaries
with open('/content/drive/My Drive/encoded_summaries.pkl', 'wb') as f:
    pickle.dump(encoded_summaries, f)

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
def compute_distance_matrix(encoded_texts):
    distance_matrix = cdist(encoded_texts, encoded_texts, metric='euclidean')
    return distance_matrix


In [ ]:
def k_center_greedy(distance_matrix, num_centers):
    n = distance_matrix.shape[0]
    centers = [np.random.randint(0, n)]
    min_distances = np.full(n, np.inf)

    for _ in range(num_centers - 1):
        min_distances = np.minimum(min_distances, distance_matrix[:, centers[-1]])
        new_center = np.argmax(min_distances)
        centers.append(new_center)

    return centers


In [ ]:
distance_matrix = compute_distance_matrix(np.vstack(encoded_texts))

In [ ]:
# Compute distance matrix
# 70, 50, 30, 10, 5, 3, 1

def getKCenterSummaries(percent):
    num_centers = math.ceil(percent*(len(texts)))
    selected_indices = k_center_greedy(distance_matrix, num_centers)
    selected_texts = [texts[i] for i in selected_indices]
    selected_summaries = [summaries[i] for i in selected_indices]
    return selected_texts, selected_summaries

percents = [1, 0.70, 0.50, 0.10, 0.05, 0.01]
selected_summaries=[]
selected_texts = []

for i in percents:
    t, s = getKCenterSummaries(i)
    selected_summaries.append(s)
    selected_texts.append(t)


print('original: ' + str(len(texts)))
for i in selected_summaries:
    print('new: :' + str(len(i)))


original: 14436
new: :14436
new: :10106
new: :7218
new: :1444
new: :722
new: :145


In [ ]:
# END OF CORESET SELECTION

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
def tokenize_function(examples):
    # Tokenize the inputs and labels
    tokenized_inputs = tokenizer(examples["texts"], padding=True, truncation=True, max_length=1024)
    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(examples["summaries"], padding=True, truncation=True, max_length=256)

    tokenized_inputs["labels"] = tokenized_labels["input_ids"]
    return tokenized_inputs

In [ ]:
def train_model(model, train_dataset, eval_dataset, tokenizer, percent):
    start = time.time()
    # we need to standardize the training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=3,
        predict_with_generate=True,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    trainer.train()
    end = time.time()
    elapsed_time = end - start
    evaluation_results = trainer.evaluate()

    model_output_dir = '/content/drive/My Drive/t5finekcenterAESLC _' + str(percent)[-2:]
    tokenizer_output_dir = '/content/drive/My Drive/t5finekcenterAESLC _' + str(percent)[-2:]
    model.save_pretrained(model_output_dir)
    tokenizer.save_pretrained(tokenizer_output_dir)

    return time, evaluation_results, model_output_dir, tokenizer_output_dir

In [ ]:
for i in range(len(selected_summaries)):
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    summary = selected_summaries[i]
    text = selected_texts[i]
    dataset = Dataset.from_dict({"texts": text, "summaries": summary})
    total_examples = len(dataset)
    print('total examples: ' + str(len(dataset)))

    train_size = int(0.95 * total_examples)
    eval_and_test_size = (total_examples - train_size)
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    train_dataset = tokenized_datasets.select(range(train_size))
    eval_dataset = tokenized_datasets.select(range(train_size, train_size+eval_and_test_size))

    print(len(train_dataset))
    print(len(eval_dataset))

    time, evaluation_results, model_output_dir, tokenizer_output_dir = train_model(model, train_dataset, eval_dataset, tokenizer, percents[i])

    print(time)
    print(evaluation_results)
    print(model_output_dir)
    print(tokenizer_output_dir)
    print('-'*100)

total examples: 14436


Map:   0%|          | 0/14436 [00:00<?, ? examples/s]

13714
722


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).